In [2]:
import pandas as pd
import os
import time
import numpy as np
import torch
import gc


In [3]:
import seaborn as sns
from scipy import stats
from statistics import NormalDist
import matplotlib.pyplot as plt

In [4]:
torch.__version__

'1.12.1+cu113'

In [5]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
#test
x = torch.rand(5, 3, 
               device=device#'cpu'
               )
print(x)

tensor([[0.8523, 0.7189, 0.1888],
        [0.4532, 0.9695, 0.1338],
        [0.8879, 0.3797, 0.7802],
        [0.3420, 0.1448, 0.5591],
        [0.5354, 0.4235, 0.6895]], device='cuda:0')


In [7]:
x.get_device()

0

In [8]:
#Deleting tensor to free memory
del x
torch.cuda.empty_cache()

In [9]:
torch.cuda.memory_allocated()

0

In [10]:
! nvidia-smi

Mon Feb 20 23:24:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  Off  | 00000000:61:00.0 Off |                    0 |
| N/A   31C    P0    35W / 250W |    994MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Logits Extractor
>
> Extracting Tensor Logits from a given Neural Code Model @danaderp
>

In [12]:
#Available Datasets
# Case1: codesearch_tesbed_EleutherAI-gpt-neo-125M_10000 for the model 'EleutherAI/gpt-neo-125M' 
# Case2: codesearch_tesbed_EleutherAI-gpt-neo-2.7B_10000 for the model 'EleutherAI/gpt-neo-2.7B' <-- MEMORY CONSTRAINTS

def params(): 
    
    code_models = {
        'Case3':('EleutherAI/gpt-neo-1.3B','codesearch_tesbed_EleutherAI-gpt-neo-1.3B_10000.csv','EleutherAI-gpt-neo-1.3B_10000_','callbacks-EleutherAI-gpt-neo-1.3B_10000_'),
        'Case4':('microsoft/CodeGPT-small-py','codesearch_tesbed_microsoft-CodeGPT-small-py_1024_10000.csv','CodeGPT-small-py_10000_','callbacks-CodeGPT-small-py_10000_'),
        'Case5':('microsoft/CodeGPT-small-py-adaptedGPT2','codesearch_tesbed_microsoft-CodeGPT-small-py-adaptedGPT2_1024_10000.csv','CodeGPT-small-py-adaptedGPT2_10000_','callbacks-CodeGPT-small-py-adaptedGPT2_10000_'),
        'Case6':('Salesforce/codegen-2B-multi','codesearch_tesbed_Salesforce-codegen-2B-multi_10000.csv','Salesforce-codegen-2B-multi_10000_','callbacks-Salesforce-codegen-2B-multi_10000_')
    }
    current_case = 'Case5' #<----[Hyper]
    
    #print(code_models[current_case][1])
    
    return {
            'big_table_path' : '../data/concept_tables/' + code_models[current_case][1],
            'hf_model' :  code_models[current_case][0],
            'model_name': code_models[current_case][2],
            'callbacks' : '../data/' + code_models[current_case][3],
            'wpe':1024  #<----[Hyper]
}

In [13]:
#pwd
parameters = params()
parameters['big_table_path']

'../data/concept_tables/codesearch_tesbed_microsoft-CodeGPT-small-py-adaptedGPT2_1024_10000.csv'

# Data Upload

In [14]:
data_pd = pd.read_csv( 
                      parameters['big_table_path'] , 
                      index_col=0
            )

In [15]:
data_pd.describe()

,model_total_input_ids
count,10000.000000
mean,334.360400
std,224.736459
min,43.000000
25%,161.000000
50%,268.000000
75%,451.000000
max,1023.000000


In [16]:
data_pd.head(5)

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids
0,"def register_context_middleware(self, *middlew...","[('def', 'def', 'function_definition'), ('regi...","[(4299, 'def', 'function_definition'), (7881, ...","[4299, 7881, 62, 22866, 62, 27171, 1574, 7, 94...",140
1,"def parse_iso_8601_time_str(time_str):\n """"...","[('def', 'def', 'function_definition'), ('pars...","[(4299, 'def', 'function_definition'), (21136,...","[4299, 21136, 62, 26786, 62, 4521, 486, 62, 24...",292
2,"def load_client_ca(self, cafile):\n """"""...","[('def', 'def', 'function_definition'), ('load...","[(4299, 'def', 'function_definition'), (3440, ...","[4299, 3440, 62, 16366, 62, 6888, 7, 944, 11, ...",241
3,"def midi_to_note(midi, octave=True, cents=Fals...","[('def', 'def', 'function_definition'), ('midi...","[(4299, 'def', 'function_definition'), (3095, ...","[4299, 3095, 72, 62, 1462, 62, 11295, 7, 13602...",871
4,"def validate_response_type(self, client_id, re...","[('def', 'def', 'function_definition'), ('vali...","[(4299, 'def', 'function_definition'), (26571,...","[4299, 26571, 62, 26209, 62, 4906, 7, 944, 11,...",242


# Model Upload

In [20]:
#tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-2B-mono")
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
#generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')
#model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")

def testing_1():
    #! pip install transformers
    from transformers import AutoTokenizer, AutoModelForCausalLM
    from transformers import pipeline
    # This is for 'EleutherAI/gpt-neo-125M'
    tokenizer = AutoTokenizer.from_pretrained(parameters['hf_model'])
    generator = pipeline(
        'text-generation', 
        model= parameters['hf_model'] )
    
    #TEST: Example 1: generation
    generator(
        "EleutherAI has", 
        do_sample=True, 
        max_new_tokens=20, 
        pad_token_id = tokenizer.eos_token_id
        )
    
    #TEST: Example 2: generation

    generator( 
        data_pd.whole_func_string.values[0][:100], #<-- Code data
        do_sample=True, 
        max_new_tokens=20,
        pad_token_id = tokenizer.eos_token_id 
        )

# Extracting Logits From a Given Model

In [21]:
#This code works for GPT-Neo
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
model = GPTNeoForCausalLM.from_pretrained( parameters['hf_model'] )
tokenizer = GPT2Tokenizer.from_pretrained( parameters['hf_model'] )

In [17]:
#This code works for CodeGPT and Salesforce
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(parameters['hf_model'])
model = AutoModelForCausalLM.from_pretrained(parameters['hf_model'])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
prompts = data_pd.whole_func_string.values

In [20]:
#Making space for the tensors
input_ids_list = tokenizer.batch_encode_plus( list(prompts) ) #<-- Do not return as a Tensor
#input_ids_list = tokenizer.batch_encode_plus( prompts , return_tensors="pt") #<-- "pt" returns as a Tensor

In [21]:
#Casting Integers to Tensor Integers. Make sure the tesor is created in a device
#We ignored the parameter attention_mask since we are not using masking here [https://huggingface.co/transformers/v4.10.1/glossary.html#attention-mask]

#input_ids_list = [torch.Tensor( np.array( input_ids ) ) for input_ids in input_ids_list.input_ids if len(input_ids) <= parameters['wpe']]
#input_ids_list = [ input_ids for input_ids in input_ids_list.input_ids if len(input_ids) <= parameters['wpe']]
input_ids_list = [torch.tensor(  input_ids, dtype = torch.int, device=device ) for input_ids in input_ids_list.input_ids if len(input_ids) <= parameters['wpe']]

In [22]:
#It should be same size
assert len(input_ids_list) == len(prompts)

In [25]:
#model.to( 'cpu' ) #WARNING, 
model.to( device ) #WARNING, Verify the device before assigning to memory

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [26]:
parameters #Verification Point of Parameters

{'big_table_path': '../data/concept_tables/codesearch_tesbed_microsoft-CodeGPT-small-py-adaptedGPT2_1024_10000.csv',
 'hf_model': 'microsoft/CodeGPT-small-py-adaptedGPT2',
 'model_name': 'CodeGPT-small-py-adaptedGPT2_10000_',
 'callbacks': '../data/callbacks-CodeGPT-small-py-adaptedGPT2_10000_',
 'wpe': 1024}

In [27]:
def logit_extractor(batch, input, from_index=0):
    """
    Output is the class CausalLMOutputWithPast (https://huggingface.co/transformers/v4.10.1/main_classes/output.html?highlight=causallmoutputwithpast)"
    logits (torch.FloatTensor of shape (batch_size, sequence_length, config.vocab_size)) – Prediction scores of the language modeling head (scores for each vocabulary token before SoftMax).
    The expression i.type(torch.LongTensor).to(device) is for casting labels for the loss
    """
    #Output is in CausalLMOutputWithPast

    for idx, n in enumerate( range( from_index, len(input), batch) ):
        output = [ model( 
            input_ids = i, 
            labels = i.type(torch.LongTensor).to(device) 
            ) for i in input[n:n+batch] ] #Labels must be provided to compute loss
    
        output_logits = [ o.logits.detach().to('cpu').numpy() for o in output ]  #Logits Extraction
        output_loss = np.array([ o.loss.detach().to('cpu').numpy() for o in output ])  #Language modeling loss (for next-token prediction).

        #Saving Callbacks
        current_batch = idx + (from_index//batch)
        for jdx, o_logits in enumerate( output_logits ):
            np.save( parameters['callbacks']+ '/'+parameters['model_name']  + f'_logits_tensor[{jdx+n}]_batch[{current_batch}].npy', o_logits)
        np.save( parameters['callbacks']+ '/'+parameters['model_name']+f'_loss_batch[{current_batch}].npy', output_loss)
        
        print(f"Batch [{current_batch}] Completed")
        
        #Memory Released
        for out in output:
            del out.logits
            torch.cuda.empty_cache()
            del out.loss
            torch.cuda.empty_cache()
        for out in output_logits:
            del out
            torch.cuda.empty_cache()
        for out in output_loss:
            del out
            torch.cuda.empty_cache()
    
    pass

In [28]:
## ACTUAL EXPERIMENT
## TIME AND MEMORY CONSUMING
logit_extractor(batch = 1, input= input_ids_list, from_index=0)

Batch [0] Completed
Batch [1] Completed
Batch [2] Completed
Batch [3] Completed
Batch [4] Completed
Batch [5] Completed
Batch [6] Completed
Batch [7] Completed
Batch [8] Completed
Batch [9] Completed
Batch [10] Completed
Batch [11] Completed
Batch [12] Completed
Batch [13] Completed
Batch [14] Completed
Batch [15] Completed
Batch [16] Completed
Batch [17] Completed
Batch [18] Completed
Batch [19] Completed
Batch [20] Completed
Batch [21] Completed
Batch [22] Completed
Batch [23] Completed
Batch [24] Completed
Batch [25] Completed
Batch [26] Completed
Batch [27] Completed
Batch [28] Completed
Batch [29] Completed
Batch [30] Completed
Batch [31] Completed
Batch [32] Completed
Batch [33] Completed
Batch [34] Completed
Batch [35] Completed
Batch [36] Completed
Batch [37] Completed
Batch [38] Completed
Batch [39] Completed
Batch [40] Completed
Batch [41] Completed
Batch [42] Completed
Batch [43] Completed
Batch [44] Completed
Batch [45] Completed
Batch [46] Completed
Batch [47] Completed
Ba

In [35]:
#logit_extractor(batch =2, input= input_ids_list[:2]) #<---- [WARNING TIME AND MEMORY CONSUMING]

In [36]:
#output_logits = np.load('../data/callbacks/logits_tensor[0]_batch[0].npy')

In [37]:
#assert output_logits.shape[0] == len(input_ids_list[0])

In [38]:
#output_loss = np.load('../data/callbacks/loss_batch[0].npy')

In [39]:
#output_loss